<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_LSTM/blob/main/AR_LSTM_tf2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun, JANG
# limitsinx.tistory.com

import numpy as np
import pandas as pd
import tensorflow as tf
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
tf.random.set_seed(1)


#Data Load
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/train_v.txt')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Batt/test_v.txt')
train_data = df_train.iloc[:,:]
test_data = df_test.iloc[:,:]
ar_train_data = df_train
ar_test_data = df_test
testify = len(ar_test_data)/2
testify = np.int(testify)
print(testify)
ar_train_data_X = df_train.iloc[:len(ar_train_data):,-3]
ar_train_data_Y = df_train.iloc[:len(ar_train_data):,-3]
ar_test_data_X = df_test.iloc[:len(ar_test_data),-3]
ar_test_data_Y = df_test.iloc[:len(ar_test_data):,-3]

ar_test_data_X_testify = df_test.iloc[:testify,-3]
ar_test_data_Y_testify = df_test.iloc[testify:,-3]
ar_test_data_X_testify_original = ar_test_data_X_testify
ar_test_data_Y_testify_original = ar_test_data_Y_testify

print(ar_test_data_X_testify)
print(ar_test_data_Y_testify)
print(testify)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Normalization
def MinMaxScaler(data):
  denom = np.max(data,0) - np.min(data,0)
  nume = data-np.min(data,0)
  return nume/denom

def back_MinMax(data,max,min):
  diff = max-min
  back = data * diff + min
  return back

In [ ]:
#Hyper parameters
seqLength = 30
dataDim = df_test.shape[1]
outputDim = 1
hiddenDim = 10
lr = 0.002
iterations = 30
batch_size = 16

trainX_max = np.max(ar_train_data_X)
trainY_max = np.max(ar_train_data_Y)
trainX_min = np.min(ar_train_data_X)
trainY_min = np.min(ar_train_data_Y)

testX_max = np.max(ar_test_data_X)
testY_max = np.max(ar_test_data_Y)
testX_min = np.min(ar_test_data_X)
testY_min = np.min(ar_test_data_Y)

ar_train_data_X = MinMaxScaler(ar_train_data_X)
ar_train_data_Y = MinMaxScaler(ar_train_data_Y)

ar_test_data_X = MinMaxScaler(ar_test_data_X)
ar_test_data_Y = MinMaxScaler(ar_test_data_Y)

ar_test_data_X_testify = ar_test_data_X[:testify]
ar_test_data_Y_testify = ar_test_data_Y[testify:]

#print(ar_test_data_X_testify.shape)
#print(ar_test_data_Y_testify.shape)

#print(testY_max)

print(ar_train_data_X.shape)
print(ar_train_data_X)
print(ar_test_data_X.shape)
print(testX_max)

In [ ]:
#Data Windowing
def sliding_window(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length - 1) : 
        if i%1000 == 0 : print(i)
        _x = time_series.iloc[i:i + seq_length,]
        _y = time_series.iloc[i + seq_length + 1 , ]  
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = sliding_window(ar_train_data_X, seqLength)
testX, testY = sliding_window(ar_test_data_X, seqLength)

print("Finished")


In [ ]:
## Stacked LSTM Modeling
# Return Sequences is very important for stacking
# This means weather output is Sequence of this layer or all of sequences
# Activation function "tanh" : Glorot normal initializer
# Activation function "reLU" : He Initiailizer
from tensorflow import keras
from tensorflow.keras import layers

initializer = tf.keras.initializers.GlorotNormal()
input_dim = 1
output_dim = input_dim
model = keras.Sequential()
model.add(layers.LSTM(units = hiddenDim, activation='tanh',input_shape = (seqLength,input_dim), return_sequences = True, kernel_initializer = initializer))
model.add(layers.LSTM(units = hiddenDim, activation='tanh', return_sequences = False, kernel_initializer = initializer))
model.add(layers.Dense(outputDim, activation='linear'))
adam = keras.optimizers.Adam(lr = lr)
model.compile(optimizer=adam, loss='mse')
model.summary()


In [ ]:
# Training
print(trainX.shape)
print(trainY.shape)
hist = model.fit(trainX, trainY, epochs = iterations, batch_size = batch_size)

In [ ]:
# Evaluate Testset
res = model.evaluate(testX, testY, batch_size = batch_size)
print(res)
#res_batch_size1 = model.evaluate(testX, testY, batch_size = 1) #Very Slow


In [ ]:
# Estimating
xhat = testX
print("Xhat : ",xhat.shape)
yhat = model.predict(xhat)
print("Yhat : ",yhat.shape)
print(testY)
print(yhat)

print("Evaluate : {}".format(np.sqrt(np.average((yhat - testY)**2))))

In [ ]:
#Estimating
estimated = back_MinMax(yhat,testX_max,testX_min) 
real = back_MinMax(testY, testY_max,testY_min)

In [ ]:
plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Stacked LSTM")
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})

print("Estimated : ",estimated.shape,estimated)
print("Real : ",real.shape, real)

plt.figure()
plt.plot(estimated[:len(estimated)],label = "Estimated by Stacked LSTM")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(real[:len(real)], label = "Real")
plt.legend(prop = {'size':10})


In [ ]:
#Forecasting
forecasting_time = 3000
#input = ar_test_data_X_testify[len(ar_test_data_X_testify) - seqLength:len(ar_test_data_X_testify)]
input = ar_test_data_X_testify[testify - seqLength:testify]
input = input.values.reshape(1,-1)
new1 = input
new_feedback1 = new1
new_feedback = new_feedback1
print("Testify : ",testify)
print(input)

#print("Testify : ",testify)
#TT = ar_test_data_X_testify_original[len(ar_test_data_X_testify) - seqLength:len(ar_test_data_X_testify)]
#print("TT : ",TT)
forecasted = []

def forecast(forecasting_time) : 
  global forecasted
  global new_feedback
  for i in range(0, forecasting_time) : 

    new = model.predict(new_feedback)
    #print("new_feedback_before : ",new_feedback)

    for j in range(1,seqLength) : 
      new_feedback[0,j-1] = new_feedback[0,j] 

    new_feedback[0,seqLength-1] = new
    #print("new_feedback_after : ",new_feedback)

    forecasted = np.append(forecasted, new)
    #print(i,"Next")
  return np.array(forecasted)

forecasting_result = forecast(forecasting_time)
#print("forecasting_result : ",forecasting_result)
forecasting_result = back_MinMax(forecasting_result,testX_max, testX_min)
print("A :",forecasting_result)
estimated = forecasting_result
#print(estimated)

In [ ]:
real = ar_test_data_Y_testify_original
reals = real[2 : forecasting_time + 2]
plt.figure()
plt.plot(reals, label = "Real")
plt.legend(prop = {'size':10})

plt.figure()
plt.plot(estimated[:len(estimated)],label = "Forecasted by LSTM")
plt.legend(prop = {'size':10})
print("Real : ",reals)
print("Foreacsting : ",estimated)

error = estimated - reals
plt.figure()
plt.plot(error,label = "Error")
plt.legend(prop = {'size':10})